In [1]:
import pickle
import pandas as pd
import torch

查看原始的数据集是什么样子的

In [2]:
directory="./NYT/"
train_txt="train.txt"
dev_txt="dev.txt"
test_txt="test.txt"
with open(directory+train_txt,"rb") as f:
    corpus_index_train0=pickle.load(f)

In [3]:
len(corpus_index_train0),corpus_index_train0

(140331,
 [[[('agrees', 'agree', 'subj', 'coll', None, None),
    ('is', 'be', 'subj', 'coll', 'alive', None),
    ('breaking', 'break', 'subj', 'coll', 'back', None),
    ('puncturing', 'puncture', 'subj', 'coll', 'lung', None),
    ('tell', 'tell', 'subj', 'coll', 'visitor', None),
    ('suffered', 'suffer', 'subj', 'coll', 'year', None),
    ('soaked', 'soak', 'subj', 'coll', None, None),
    ('hoist', 'hoist', 'subj', 'coll', 'coll', 'wheelchair')],
   [('looking', 'look', 'subj', 'coll', None, 'improvement'),
    ('introduce', 'introduce', 'subj', 'coll', 'enquirer', None),
    ('likes', 'like', 'subj', 'coll', 'shepard', None),
    ('discussed', 'discuss', 'subj', 'coll', 'specifics', None),
    ('told', 'tell', 'subj', 'coll', '`eric', None)],
   2],
  [[('spent', 'spend', 'subj', 'barden', '$', None),
    ('recovered', 'recover', 'subj', 'barden', None, None),
    ('added', 'add', 'subj', 'barden', None, None),
    ('focusing', 'focus', 'subj', 'barden', None, 'business'),
    

In [5]:
with open(directory+dev_txt,"rb") as f:
    corpus_index_dev=pickle.load(f)
with open(directory+test_txt,"rb") as f:
    corpus_index_test=pickle.load(f)

拿出每个句子的第一个单词并且做成一个字典，每个单词不重复

In [6]:
corpus_list=[]
# 选择处理的数据的源 （训练集、验证集、测试集）
for source in [corpus_index_train0,corpus_index_dev,corpus_index_test]:
    # 遍历所有的序列+候选组合
    for i in range(len(source)):
        # 分别遍历每个组合的序列和候选
        for j in range(2):
            # 遍历序列
            if j==0:
                for k in range(8):
                    # 这里不用第一个单词而是用整个句子
                    corpus_list.append(source[i][j][k][0:3:2])#[0].replace("'","")
            # 遍历候选
            if j==1:
                for k in range(5):
                    corpus_list.append(source[i][j][k][0:3:2])#[0].replace("'","")
len(corpus_list),corpus_list[:100]

(2084303,
 [('agrees', 'subj'),
  ('is', 'subj'),
  ('breaking', 'subj'),
  ('puncturing', 'subj'),
  ('tell', 'subj'),
  ('suffered', 'subj'),
  ('soaked', 'subj'),
  ('hoist', 'subj'),
  ('looking', 'subj'),
  ('introduce', 'subj'),
  ('likes', 'subj'),
  ('discussed', 'subj'),
  ('told', 'subj'),
  ('spent', 'subj'),
  ('recovered', 'subj'),
  ('added', 'subj'),
  ('focusing', 'subj'),
  ('see', 'obj'),
  ('was', 'subj'),
  ('seek+out', 'subj'),
  ('maximizing', 'subj'),
  ('added', 'subj'),
  ('aided', 'subj'),
  ('led', 'subj'),
  ('use', 'obj'),
  ('seems', 'subj'),
  ('blew', 'subj'),
  ('save', 'subj'),
  ('surrendered', 'subj'),
  ('summoned', 'obj'),
  ('gave+up', 'subj'),
  ('caught', 'subj'),
  ('hung', 'subj'),
  ('looked', 'subj'),
  ('travels', 'subj'),
  ('traveled', 'subj'),
  ('happened', 'prep_to'),
  ('opened', 'subj'),
  ('escape', 'subj'),
  ('left', 'obj'),
  ('setting', 'obj'),
  ('sighs', 'subj'),
  ('retreated', 'subj'),
  ('tell', 'subj'),
  ('became', 'subj'

In [11]:
with open(directory+"corpus_verb_mapping.pkl","wb") as f:
    pickle.dump(set(corpus_list),f)
with open(directory+"corpus_verb_mapping.pkl","rb") as f:
    corpus_verb_mapping=pickle.load(f)
len(corpus_verb_mapping),corpus_verb_mapping

(47256,
 {('bog+down', 'subj'),
  ('roll', 'subj'),
  ('yawn', 'subj'),
  ('investing', 'subj'),
  ('discussing', 'subj'),
  ('bases', 'subj'),
  ('ushered', 'obj'),
  ('ratified', 'subj'),
  ('shut+out', 'obj'),
  ('withdrew', 'subj'),
  ('had', 'prep_with'),
  ('setting+off', 'obj'),
  ('acquire', 'prep_to'),
  ('solidify', 'subj'),
  ('Built', 'prep_as'),
  ('heat', 'subj'),
  ('defers', 'subj'),
  ('crossed', 'obj'),
  ('split', 'prep_from'),
  ('stared', 'subj'),
  ('Arm', 'subj'),
  ('hams+up', 'obj'),
  ('clouds', 'subj'),
  ('tripped+up', 'subj'),
  ('saved', 'prep_from'),
  ('reauthorize', 'obj'),
  ('jousting', 'subj'),
  ('repriced', 'subj'),
  ('eats+up', 'obj'),
  ('slugging+out', 'obj'),
  ('snowballs', 'subj'),
  ('featuring', 'obj'),
  ('omitted', 'obj'),
  ('blotted', 'subj'),
  ('ruling+out', 'obj'),
  ('hatred', 'obj'),
  ('feel', 'prep_than'),
  ('refitted', 'subj'),
  ('meander', 'subj'),
  ('billed', 'prep_for'),
  ('boycott', 'obj'),
  ('looking+up', 'prep_as'),


In [10]:
corpus_verb_mapping.index("伪造"),len(corpus_verb_mapping)

(19, 219)

In [15]:
corpus_index_train0[i][j][k][0:3:2]

('proposed', 'subj')

处理事件之间的序列关系，并且保存

In [16]:
# 集合转列表
corpus_verb_mapping_list=list(corpus_verb_mapping)

# 选择处理的数据的源 （训练集、验证集、测试集）
for source in [corpus_index_train0,corpus_index_dev,corpus_index_test]:
    event_sequence_list=[]
    # 遍历所有的序列+候选组合
    for i in range(len(source)):
        cur_sequence_list=[]
        # 分别遍历每个组合的序列和候选
        for j in range(2):
            # 遍历序列
            if j==0:
                for k in range(8):
                    cur_sequence_list.append(corpus_verb_mapping_list.index(source[i][j][k][0:3:2]))#.replace("'","")
            # 遍历候选
            if j==1:
                target=source[i][2]
                cur_sequence_list.append(corpus_verb_mapping_list.index(source[i][j][target][0:3:2]))#.replace("'","")
        event_sequence_list.append(cur_sequence_list)
    if source==corpus_index_train0:
        with open(directory+"train_sequence.pkl","wb") as f:
            pickle.dump(event_sequence_list,f)
    elif source==corpus_index_dev:
        with open(directory+"dev_sequence.pkl","wb") as f:
            pickle.dump(event_sequence_list,f)
    else:
        with open(directory+"test_sequence.pkl","wb") as f:
            pickle.dump(event_sequence_list,f)

In [12]:
corpus_index_dev

[[[('亏损', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('亏损', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('举报', 'None', '发生', '负面报道', '长园集团', '客体公司'),
   ('举报', 'None', '发生', '负面报道', '长园集团', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('低于', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('净利润下降', 'None', '发生', '业务异常', '长园集团', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '长园集团', '客体公司')],
  [('低于', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('整改', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('低于', 'None', '发生', '经营异常', '寒锐钴业', '客体公司'),
   ('整改', 'None', '发生', '经营异常', '长园集团', '客体公司'),
   ('低于', 'None', '发生', '违法违规', '五粮液', '客体公司')],
  0],
 [[('低于', 'None', '发生', '经营异常', '金刚玻璃', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('低于', 'None', '发生', '经营异常', '金刚玻璃', '客体公司'),
   ('同比下降', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('资产减值', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('资产减值', 'None', '发生', '业务异常', '金刚玻璃', '客体公司'),
   ('立案', 'None', '发生', '违法违规', '金刚玻璃', '客体公司')

构建对应的知识图

In [4]:
import dgl
import numpy as np

In [10]:
with open(directory+"./company_event/train_sequence.pkl","rb") as f:
    train_sequence=pickle.load(f)
with open(directory+"dev_sequence.pkl","rb") as f:
    dev_sequence=pickle.load(f)
with open(directory+"test_sequence.pkl","rb") as f:
    test_sequence=pickle.load(f)

print(len(train_sequence),len(dev_sequence),len(test_sequence))

src_dst=np.empty(shape=(8*(len(train_sequence)+len(dev_sequence)+len(test_sequence)),2),dtype=np.long)
row=0
for source in [train_sequence,dev_sequence,test_sequence]:
    for i in range(len(source)):
        for j in range(8):
            src_dst[row,0]=source[i][j]
            src_dst[row,1]=source[i][j+1]
            row=row+1
np.save("./company_event/src_dst.npy",src_dst)


1600 500 240


C:\Users\Weirdo\AppData\Local\Temp\ipykernel_2524\4062996827.py:10: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  src_dst=np.empty(shape=(8*(len(train_sequence)+len(dev_sequence)+len(test_sequence)),2),dtype=np.long)


保存事件之间的关系图

In [12]:
import torch
src_dst=torch.from_numpy(src_dst)
src=src_dst[:,0]
dst=src_dst[:,1]
# g=dgl.graph(data=(src,dst),num_nodes=len(corpus_verb_mapping))
# dgl.save_graphs("data/dataset/NYT/kg/graph.bin",[g])

构建DGL-KE需要的五个文件，进行transE的训练

In [15]:
import pandas as pd
src_dst_dataframe=pd.DataFrame(columns=["src","dst"],data=src_dst,dtype=int)
src_dst_train=src_dst_dataframe.iloc[:row*2//3,:]
src_dst_valid=src_dst_dataframe.iloc[row*2//3:row*8//9,:]
src_dst_test=src_dst_dataframe.iloc[row*2//9:,:]

In [16]:
src_dst_train.iloc[:5],src_dst_valid[:5],src_dst_test[:5]

(   src  dst
 0  132   20
 1   20  142
 2  142  104
 3  104   20
 4   20   38,
        src  dst
 12480   73  115
 12481  115   20
 12482   20   20
 12483   20   20
 12484   20   20,
       src  dst
 4160   11  160
 4161  160   38
 4162   38  141
 4163  141   73
 4164   73  104)

In [18]:
src_dst_train.to_csv("./company_event/kg/src_dst_train.csv",header=False,index=False)
src_dst_valid.to_csv("./company_event/kg/src_dst_val.csv",header=False,index=False)
src_dst_test.to_csv("./company_event/kg/src_dst_test.csv",header=False,index=False)

In [9]:
verb_dict=dict(zip(range(len(corpus_verb_mapping)),corpus_verb_mapping))
with open("./company_event/verb_dict.dict","w",encoding="utf-8") as f:
    for k,v in verb_dict.items():
        f.write(str(k)+","+v)
        f.write("\n")

In [2]:
import pandas as pd
import pickle
with open("data/dataset/NYT/train_sequence.pkl","rb") as f:
    train_sequence=pickle.load(f)
with open("data/dataset/NYT/dev_sequence.pkl","rb") as f:
    dev_sequence=pickle.load(f)
with open("data/dataset/NYT/test_sequence.pkl","rb") as f:
    test_sequence=pickle.load(f)
src_dst=pd.read_csv("data/dataset/NYT/kg/src_dst.csv")
src_dst_train=src_dst.iloc[:8*len(train_sequence),:]
src_dst_valid=src_dst.iloc[:8*len(dev_sequence),:]
src_dst_test=src_dst.iloc[:8*len(test_sequence),:]
src_dst_train.to_csv("./company_event/src_dst_train.csv",header=False,index=False)
src_dst_valid.to_csv("./company_event/src_dst_val.csv",header=False,index=False)
src_dst_test.to_csv("./company_event/src_dst_test.csv",header=False,index=False)

In [3]:
import pickle
with open("./company_event/dev_sequence.pkl","rb") as f:
    dev_seq=pickle.load(f)

dev_seq[:5]

[[11, 11, 10, 10, 73, 20, 44, 73, 20],
 [20, 73, 20, 73, 168, 168, 137, 47, 73],
 [38, 11, 160, 20, 38, 38, 20, 20, 38],
 [50, 11, 20, 104, 211, 115, 160, 11, 47],
 [11, 11, 11, 11, 160, 11, 11, 170, 11]]